In [0]:
confluentBootstrapServers = 'pkc-l7pr2.ap-south-1.aws.confluent.cloud:9092'
confluentApiKey = 'Z2OSAATSA55GQVZK'
confluentSecret = 'a1QC+319eHmIkhcb665BX6v2PCI5utizHlkPmZJQX43Tbx0ZtlWEht4888R1IJqq'
confluentTopicName = 'retail-data-new'

In [0]:
orders_df = spark \
.readStream \
.format("kafka") \
.option("kafka.bootstrap.servers",confluentBootstrapServers) \
.option("kafka.security.protocol","SASL_SSL") \
.option("kafka.sasl.mechanism","PLAIN") \
.option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret)) \
.option("kafka.ssl.endpoint.identification.algorithm","https") \
.option("subscribe",confluentTopicName) \
.option("startingTimestamp",1) \
.option("maxOffsetsPerTrigger",50) \
.load()


In [0]:
converted_orders_df = orders_df.selectExpr("CAST(key as string) AS key","CAST(value as string) AS value","topic","partition","offset","timestamp","timestampType")

In [0]:
display(converted_orders_df)

key,value,topic,partition,offset,timestamp,timestampType
256,"{""order_id"":2,""customer_id"":256,""customer_fname"":""David"",""customer_lname"":""Rodriguez"",""city"":""Chicago"",""state"":""IL"",""pincode"":60625,""line_items"":[{""order_item_id"":2,""order_item_product_id"":1073,""order_item_quantity"":1,""order_item_product_price"":199.99,""order_item_subtotal"":199.99},{""order_item_id"":3,""order_item_product_id"":502,""order_item_quantity"":5,""order_item_product_price"":50.0,""order_item_subtotal"":250.0},{""order_item_id"":4,""order_item_product_id"":403,""order_item_quantity"":1,""order_item_product_price"":129.99,""order_item_subtotal"":129.99}]}",retail-data-new,1,0,2024-04-25T07:20:39.135+0000,0
4530,"{""order_id"":7,""customer_id"":4530,""customer_fname"":""Mary"",""customer_lname"":""Smith"",""city"":""Miami"",""state"":""FL"",""pincode"":33161,""line_items"":[{""order_item_id"":14,""order_item_product_id"":1073,""order_item_quantity"":1,""order_item_product_price"":199.99,""order_item_subtotal"":199.99},{""order_item_id"":15,""order_item_product_id"":957,""order_item_quantity"":1,""order_item_product_price"":299.98,""order_item_subtotal"":299.98},{""order_item_id"":16,""order_item_product_id"":926,""order_item_quantity"":5,""order_item_product_price"":15.99,""order_item_subtotal"":79.95}]}",retail-data-new,1,1,2024-04-25T07:20:40.096+0000,0
918,"{""order_id"":11,""customer_id"":918,""customer_fname"":""Nathan"",""customer_lname"":""Smith"",""city"":""Caguas"",""state"":""PR"",""pincode"":725,""line_items"":[{""order_item_id"":29,""order_item_product_id"":365,""order_item_quantity"":1,""order_item_product_price"":59.99,""order_item_subtotal"":59.99},{""order_item_id"":30,""order_item_product_id"":627,""order_item_quantity"":4,""order_item_product_price"":39.99,""order_item_subtotal"":159.96},{""order_item_id"":31,""order_item_product_id"":1014,""order_item_quantity"":1,""order_item_product_price"":49.98,""order_item_subtotal"":49.98},{""order_item_id"":32,""order_item_product_id"":191,""order_item_quantity"":4,""order_item_product_price"":99.99,""order_item_subtotal"":399.96},{""order_item_id"":33,""order_item_product_id"":1014,""order_item_quantity"":5,""order_item_product_price"":49.98,""order_item_subtotal"":249.9}]}",retail-data-new,1,2,2024-04-25T07:20:41.084+0000,0
9149,"{""order_id"":13,""customer_id"":9149,""customer_fname"":""Ronald"",""customer_lname"":""Whitehead"",""city"":""Santa Ana"",""state"":""CA"",""pincode"":92705,""line_items"":[{""order_item_id"":39,""order_item_product_id"":276,""order_item_quantity"":4,""order_item_product_price"":31.99,""order_item_subtotal"":127.96}]}",retail-data-new,1,3,2024-04-25T07:20:41.804+0000,0
7276,"{""order_id"":16,""customer_id"":7276,""customer_fname"":""Pamela"",""customer_lname"":""Smith"",""city"":""Caguas"",""state"":""PR"",""pincode"":725,""line_items"":[{""order_item_id"":48,""order_item_product_id"":365,""order_item_quantity"":2,""order_item_product_price"":59.99,""order_item_subtotal"":119.98},{""order_item_id"":49,""order_item_product_id"":365,""order_item_quantity"":5,""order_item_product_price"":59.99,""order_item_subtotal"":299.95}]}",retail-data-new,1,4,2024-04-25T07:20:42.208+0000,0
4367,"{""order_id"":23,""customer_id"":4367,""customer_fname"":""Danielle"",""customer_lname"":""Moran"",""city"":""Springfield"",""state"":""MO"",""pincode"":65807,""line_items"":[{""order_item_id"":68,""order_item_product_id"":957,""order_item_quantity"":1,""order_item_product_price"":299.98,""order_item_subtotal"":299.98}]}",retail-data-new,1,5,2024-04-25T07:20:43.150+0000,0
9776,"{""order_id"":42,""customer_id"":9776,""customer_fname"":""Diane"",""customer_lname"":""Davis"",""city"":""Endicott"",""state"":""NY"",""pincode"":13760,""line_items"":[{""order_item_id"":106,""order_item_product_id"":627,""order_item_quantity"":1,""order_item_product_price"":39.99,""order_item_subtotal"":39.99},{""order_item_id"":107,""order_item_product_id"":365,""order_item_quantity"

In [0]:
converted_orders_df \
.writeStream \
.queryName("ingestionquery") \
.format("delta") \
.outputMode("append") \
.option("checkpointLocation","checkpointdir301") \
.toTable("orderstablenew301")


Out[6]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f31e1a4c2e0>

In [0]:
spark.sql("select * from orderstablenew301").show()

+-----+--------------------+---------------+---------+------+--------------------+-------------+
|  key|               value|          topic|partition|offset|           timestamp|timestampType|
+-----+--------------------+---------------+---------+------+--------------------+-------------+
| 2911|{"order_id":8,"cu...|retail-data-new|        5|     0|2024-04-25 07:20:...|            0|
| 2711|{"order_id":21,"c...|retail-data-new|        5|     1|2024-04-25 07:20:...|            0|
| 9503|{"order_id":25,"c...|retail-data-new|        5|     2|2024-04-25 07:20:...|            0|
| 4840|{"order_id":35,"c...|retail-data-new|        5|     3|2024-04-25 07:20:...|            0|
| 8214|{"order_id":39,"c...|retail-data-new|        5|     4|2024-04-25 07:20:...|            0|
| 2636|{"order_id":45,"c...|retail-data-new|        5|     5|2024-04-25 07:20:...|            0|
| 9213|{"order_id":58,"c...|retail-data-new|        5|     6|2024-04-25 07:20:...|            0|
|11644|{"order_id":59,"c...|re